In [1]:
import postprocessing as pp
import numpy as np
import time

In [2]:
def waste_ratio(old_data, new_data):
    return len(new_data)/len(old_data)

In [3]:
# Get datasets
# dataset_tags stores names of dataset, combine with dictionary to minimize clutter
# Each dataset tag represent the type of data
dataset_size = 100000
dataset_tags = ["classical_low_data", "classical_high_data", "quantum_data"]
datasets = {}

# Manual entry of each of your datasets, ensure they match with tag
classical_low_data = pp.generate_low_entropy_array(dataset_size)
classical_high_data = pp.generate_high_entropy_array(dataset_size)
quantum_data = pp.generate_qrng_data(dataset_size)

datasets[dataset_tags[0]] = classical_low_data
datasets[dataset_tags[1]] = classical_high_data
datasets[dataset_tags[2]] = quantum_data

In [4]:
# Measure before entropy
before_entropy = {}
for tag in dataset_tags:
    before_entropy[tag] = pp.shannon_entropy(datasets[tag])

In [5]:
# Apply transformations, and measure the throughput

# Get toeplitz matrix

# block_size = 
# toeplitz_size = dataset_size
# first_row = np.random.randint(0, 2, toeplitz_size)
# first_column = np.random.randint(0, 2, toeplitz_size)
# toeplitz_matrix = pp.create_toeplitz(first_row, first_column)
neumann_datasets = {}
toeplitz_datasets = {}
FFT_datasets = {}
parity_datasets = {}
# Use average across all datasets for throughput
# Throughput is current just the time it takes
timed_throughput = [0, 0, 0, 0]
waste_ratios = [0, 0, 0, 0]

for tag in dataset_tags:
    # Manually apply each method
    t0 = time.time()
    neumann_datasets[tag] = pp.apply_von_neumann_extractor(datasets[tag])
    t1 = time.time()
    timed_throughput[0] += t1 - t0
    waste_ratios[0] += waste_ratio(datasets[tag], neumann_datasets[tag])


    t0 = time.time()
    toeplitz_datasets[tag] = pp.apply_toeplitz_transformation(datasets[tag])
    t1 = time.time()
    timed_throughput[1] += t1 - t0
    waste_ratios[1] += waste_ratio(datasets[tag], toeplitz_datasets[tag])

    t0 = time.time()
    FFT_datasets[tag] = pp.apply_fft_toeplitz(datasets[tag])
    t1 = time.time()
    timed_throughput[2] += t1 - t0
    waste_ratios[2] += waste_ratio(datasets[tag], FFT_datasets[tag])

    t0 = time.time()
    parity_datasets[tag] = pp.apply_parity_extractor(datasets[tag])
    t1 = time.time()
    timed_throughput[3] += t1 - t0
    waste_ratios[3] += waste_ratio(datasets[tag], parity_datasets[tag])

for i in range(0,4):
    timed_throughput[i] /= 4
    waste_ratios[i] /= 4

In [6]:
# Measure entropy of each applied transform
# neumann_datasets = {}
# toeplitz_datasets = {}
# FFT_datasets = {}
# parity_datasets = {}

neumann_entropy = {}
toeplitz_entropy = {}
FFT_entropy = {}
parity_entropy = {}

for tag in dataset_tags:
    neumann_entropy[tag] = pp.shannon_entropy(neumann_datasets[tag])
    toeplitz_entropy[tag] = pp.shannon_entropy(toeplitz_datasets[tag])
    FFT_entropy[tag] = pp.shannon_entropy(FFT_datasets[tag])
    parity_entropy[tag] = pp.shannon_entropy(parity_datasets[tag])

In [8]:
# Display results
print("Method\t\tP=0.9\t\tP=0.5\t\tSimulator\t\tWaste Ratio\t\tThroughput\t\t")

print("Control\t\t", end="")
for tag in dataset_tags:
    print(f"{before_entropy[tag]:.5f}\t\t", end="")
print("\t1.00000\t\t", end="")
print("\tN/A\t\t", end="")
# print(f"{timed_throughput[0]}:.5f\t\t")
print('\n')

print("Von Neumann\t", end="")
for tag in dataset_tags:
    print(f"{neumann_entropy[tag]:.5f}\t\t", end="")
print(f"\t{waste_ratios[0]:.5f}\t\t", end="")
print(f"\t{timed_throughput[0]:.5f}\t\t", end="")
print('\n')

print("Toeplitz\t", end="")
for tag in dataset_tags:
    print(f"{toeplitz_entropy[tag]:.5f}\t\t", end="")
print(f"\t{waste_ratios[1]:.5f}\t\t", end="")
print(f"\t{timed_throughput[1]:.5f}\t\t", end="")
print('\n')

print("FFT   \t\t", end="")
for tag in dataset_tags:
    print(f"{FFT_entropy[tag]:.5f}\t\t", end="")
print(f"\t{waste_ratios[2]:.5f}\t\t", end="")
print(f"\t{timed_throughput[2]:.5f}\t\t", end="")
print('\n')

print("Parity\t\t", end="")
for tag in dataset_tags:
    print(f"{parity_entropy[tag]:.5f}\t\t", end="")
print(f"\t{waste_ratios[3]:.5f}\t\t", end="")
print(f"\t{timed_throughput[3]:.5f}\t\t", end="")
print('\n')

Method		P=0.9		P=0.5		Simulator		Waste Ratio		Throughput		
Control		0.46846		0.99999		1.00000			1.00000			N/A		

Von Neumann	0.99985		0.99998		1.00000			0.14739			0.00750		

Toeplitz	0.99999		1.00000		1.00000			0.74976			0.01449		

FFT   		1.00000		1.00000		0.99996			0.74976			0.01609		

Parity		0.87134		0.99988		0.99997			0.18750			0.00367		

